In [2]:
import requests
import os
import csv
import sqlite3
import pandas as pd

__all__=['csv_to_database']

'''
執行前,需先建立datasource資料夾,底下建立age/incom/education/job/sex資料夾
'''
# ---------下載資料---------#
def __download_credit_data() -> csv:
    area = [
        "KLC",
        "TPE",
        "NTP",
        "TYC",
        "HCC",
        "HCH",
        "MLH",
        "TCC",
        "CHH",
        "NTH",
        "YUH",
        "CYC",
        "CYH",
        "TNC",
        "KHC",
        "PTH",
        "TTH",
        "HLH",
        "YIH",
        "PHH",
        "KMH",
        "LCH",
        "X1",
        "LCSUM",
        "MCT",
        "LOC",
    ]
    area_code = {
        "63000000": "臺北市",
        "64000000": "高雄市",
        "65000000": "新北市",
        "66000000": "臺中市",
        "67000000": "臺南市",
        "68000000": "桃園市",
        "10002000": "宜蘭縣",
        "10004000": "新竹縣",
        "10005000": "苗栗縣",
        "10007000": "彰化縣",
        "10008000": "南投縣",
        "10009000": "雲林縣",
        "10010000": "嘉義縣",
        "10020000": "嘉義市",
        "10013000": "屏東縣",
        "10014000": "臺東縣",
        "10015000": "花蓮縣",
        "10016000": "澎湖縣",
        "10017000": "基隆市",
        "10018000": "新竹市",
        "9020000": "金門縣",
        "9007000": "連江縣",
    }
    industry = ["FD", "CT", "LG", "TR", "EE", "DP", "X2", "OT", " IDSUM", "ALL"]
    DataType = ["sex", "job", "incom", "education", "age"]
    sex = ["M", "F"]
    sex_code = {"1": "男性", "2": "女性"}

    #---兩性消費---#
    for A in industry:
        for B in area:
            sex_url = (
                f"https://bas.nccc.com.tw/nccc-nop/OpenAPI/C01/sexconsumption/{B}/{A}"
            )
            response_sex = requests.request("GET", sex_url)
            if len(response_sex.text) == 0:
                continue
            with open(f"./datasource/sex/sex{B}_{A}.csv", "wb") as file:
                file.write(response_sex.content)
    print("性別消費資料讀取成功")

    #---各職業類別消費---#
    for E in industry:
        for F in area:
            job_url = (
                f"https://bas.nccc.com.tw/nccc-nop/OpenAPI/C04/jobsconsumption/{F}/{E}"
            )
            response_job = requests.request("GET", job_url)
            if len(response_job.text) == 0:
                continue
            with open(f"./datasource/job/job{F}_{E}.csv", "wb") as file:
                file.write(response_job.content)
    print("職業類別消費資料讀取成功")

    #---各年收入族群消費---#
    for G in industry:
        for H in area:
            incom_url = f"https://bas.nccc.com.tw/nccc-nop/OpenAPI/C03/incomegroupsconsumption/{H}/{G}"
            response_incom = requests.request("GET", incom_url)
            if len(response_incom.text) == 0:
                continue
            with open(f"./datasource/incom/incom{H}_{G}.csv", "wb") as file:
                file.write(response_incom.content)
    print("收入類別消費資料讀取成功")

    #---各教育程度消費---#
    for I in industry:
        for J in area:
            education_url = f"https://bas.nccc.com.tw/nccc-nop/OpenAPI/C05/educationconsumption/{J}/{I}"
            response_education = requests.request("GET", education_url)
            if len(response_education.text) == 0:
                continue
            with open(f"./datasource/education/education{J}_{I}.csv", "wb") as file:
                file.write(response_education.content)
    print("教育程度資料讀取成功")

    #---兩性X各年齡層消費---#
    for A in industry:
        for B in area:
            for C in sex:
                age_url = f"https://bas.nccc.com.tw/nccc-nop/OpenAPI/C11/GenderAgeGroup/{B}/{A}/{C}"
                response_age = requests.request("GET", age_url)
                if len(response_age.text) == 0:
                    continue
                folder_path = "./datasource/age/"
                file_name = f"age{B}_{A}_{C}.csv"
                file_path = os.path.join(folder_path, file_name)
                with open(file_path, "wb") as file:
                    file.write(response_age.content)
    print("年齡層消費資料讀取成功")

    # ---------合併csv---------#
    for item in DataType:
        path = f"./datasource/{item}/"
        csv_files = [file for file in os.listdir(path) if file.endswith(".csv")]
        merged_data = pd.DataFrame()
        for file in csv_files:
            file_path = os.path.join(path, file)
            data = pd.read_csv(file_path)
            merged_data = pd.concat([merged_data, data], ignore_index=True)
        merged_data.drop_duplicates(inplace=True)
        merged_data.to_csv(f"{item}.csv", index=False)
        print(f"{item}.csv建立成功")

        with open(f"./{item}.csv", "r", encoding="UTF-8") as csv_file:
            csv_reader = csv.DictReader(csv_file)
            fieldnames = csv_reader.fieldnames

            with open(f"./{item}_trans.csv", "w", encoding="utf-8", newline="") as file:
                new_fieldnames = ["年", "月"] + fieldnames[1:]
                csv_writer = csv.DictWriter(file, fieldnames=new_fieldnames)
                csv_writer.writeheader()

                for row in csv_reader:
                    #---找不到對應的值，則保持原本的值---#
                    row["地區"] = area_code.get(row["地區"], row["地區"])

                    if "性別" in fieldnames:
                        row["性別"] = sex_code.get(row["性別"], row["性別"])

                    year = row["年月"][:4]
                    month = row["年月"][4:]
                    new_row = {"年": year, "月": month, "地區": row["地區"]}

                    if "性別" in fieldnames:
                        new_row["性別"] = row["性別"]

                    new_row.update(row)
                    del new_row["年月"]
                    csv_writer.writerow(new_row)

                print(f"{item}_trans.csv建立成功")

__download_credit_data()

性別消費資料讀取成功
職業類別消費資料讀取成功
收入類別消費資料讀取成功
教育程度資料讀取成功
年齡層消費資料讀取成功
sex.csv建立成功
sex_trans.csv建立成功
job.csv建立成功
job_trans.csv建立成功
incom.csv建立成功
incom_trans.csv建立成功
education.csv建立成功
education_trans.csv建立成功
age.csv建立成功
age_trans.csv建立成功


In [3]:
# ---------輸入資料---------#
def csv_to_database() -> None:
    conn = sqlite3.connect("creditcard.db")
    DataType = ["sex", "job", "incom", "education", "age"]
    for item in DataType:
        file = f"./{item}_trans.csv"
        df = pd.read_csv(file)

        df.rename(columns={"信用卡交易金額[新台幣]": "信用卡金額"}, inplace=True)
        df.to_sql(item, conn, if_exists="replace", index=False)

    conn.close()

csv_to_database()

In [2]:
import pandas as pd

df = pd.read_csv("age_trans.csv")
df

,年,月,地區,產業別,性別,年齡層,信用卡交易筆數,信用卡交易金額[新台幣]
0,2014,1,彰化縣,衣,女性,未滿20歲,282,574794
1,2014,1,彰化縣,衣,女性,20(含)-25歲,7091,16563668
2,2014,1,彰化縣,衣,女性,25(含)-30歲,20467,52197577
3,2014,1,彰化縣,衣,女性,30(含)-35歲,35400,93635594
4,2014,1,彰化縣,衣,女性,35(含)-40歲,41102,113811652
...,...,...,...,...,...,...,...,...
504499,2023,9,臺南市,其他,男性,60(含)-65歲,5651,27495078
504500,2023,9,臺南市,其他,男性,65(含)-70歲,3589,20028488
504501,2023,9,臺南市,其他,男性,70(含)-75歲,2009,10553100
504502,2023,9,臺南市,其他,男性,75(含)-80歲,602,3561448


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504504 entries, 0 to 504503
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   年             504504 non-null  int64 
 1   月             504504 non-null  int64 
 2   地區            504504 non-null  object
 3   產業別           504504 non-null  object
 4   性別            504504 non-null  object
 5   年齡層           504504 non-null  object
 6   信用卡交易筆數       504504 non-null  int64 
 7   信用卡交易金額[新台幣]  504504 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 30.8+ MB


# 資料標準化 z-score

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

#將數據收斂
df = pd.read_csv('age_trans.csv')
std = StandardScaler()
df1 = df[['年','月','信用卡交易筆數','信用卡交易金額[新台幣]']]
arr = std.fit_transform(df1)
df[['年','月','信用卡交易筆數','信用卡交易金額[新台幣]']] = arr
df

,年,月,地區,產業別,性別,年齡層,信用卡交易筆數,信用卡交易金額[新台幣]
0,-1.557121,-1.576054,彰化縣,衣,女性,未滿20歲,-0.172363,-0.147517
1,-1.557121,-1.576054,彰化縣,衣,女性,20(含)-25歲,-0.119324,-0.093964
2,-1.557121,-1.576054,彰化縣,衣,女性,25(含)-30歲,-0.015133,0.025389
3,-1.557121,-1.576054,彰化縣,衣,女性,30(含)-35歲,0.101186,0.164183
4,-1.557121,-1.576054,彰化縣,衣,女性,35(含)-40歲,0.145601,0.231761
...,...,...,...,...,...,...,...,...
504499,1.639075,0.765512,臺南市,其他,男性,60(含)-65歲,-0.130541,-0.057350
504500,1.639075,0.765512,臺南市,其他,男性,65(含)-70歲,-0.146603,-0.082358
504501,1.639075,0.765512,臺南市,其他,男性,70(含)-75歲,-0.158910,-0.114096
504502,1.639075,0.765512,臺南市,其他,男性,75(含)-80歲,-0.169870,-0.137513


In [13]:
df.describe()


,年,月,信用卡交易筆數,信用卡交易金額[新台幣]
count,504504.000000,504504.000000,5.045040e+05,5.045040e+05
mean,2018.384615,6.384615,2.240979e+04,4.461733e+07
std,2.815850,3.416520,1.283794e+05,2.985592e+08
min,2014.000000,1.000000,-7.000000e+00,-9.275543e+07
25%,2016.000000,3.000000,1.900000e+02,4.397582e+05
50%,2018.000000,6.000000,1.351000e+03,3.267526e+06
75%,2021.000000,9.000000,6.872000e+03,1.429571e+07
max,2023.000000,12.000000,3.828945e+06,2.660613e+10


In [16]:
# Check for duplicate entries in the specified columns
duplicates = df.duplicated(subset=['年', '月'])

# Print the rows with duplicate entries
print(df[duplicates])


           年  月   地區 產業別  性別        年齡層  信用卡交易筆數  信用卡交易金額[新台幣]
1       2014  1  彰化縣   衣  女性  20(含)-25歲     7091      16563668
2       2014  1  彰化縣   衣  女性  25(含)-30歲    20467      52197577
3       2014  1  彰化縣   衣  女性  30(含)-35歲    35400      93635594
4       2014  1  彰化縣   衣  女性  35(含)-40歲    41102     113811652
5       2014  1  彰化縣   衣  女性  40(含)-45歲    37926     111612989
...      ... ..  ...  ..  ..        ...      ...           ...
504499  2023  9  臺南市  其他  男性  60(含)-65歲     5651      27495078
504500  2023  9  臺南市  其他  男性  65(含)-70歲     3589      20028488
504501  2023  9  臺南市  其他  男性  70(含)-75歲     2009      10553100
504502  2023  9  臺南市  其他  男性  75(含)-80歲      602       3561448
504503  2023  9  臺南市  其他  男性   80(含)歲以上      369       2306471

[504387 rows x 8 columns]


In [17]:
# Remove duplicates based on the specified columns
df = df.drop_duplicates(subset=['年', '月'])